In [1]:
import numpy as np

from sklearn.metrics import accuracy_score
from keras.datasets import reuters
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Activation
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier

from keras.layers import Bidirectional

Using TensorFlow backend.


In [2]:
# parameters for data load
num_words = 30000
maxlen = 50
test_split = 0.3

(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words = num_words, maxlen = maxlen, test_split = test_split)

In [3]:
# pad the sequences with zeros 
# padding parameter is set to 'post' => 0's are appended to end of sequences
X_train = pad_sequences(X_train, padding = 'post')
X_test = pad_sequences(X_test, padding = 'post')

X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))

y_data = np.concatenate((y_train, y_test))
y_data = to_categorical(y_data)

y_train = y_data[:1395]
y_test = y_data[1395:]

# 데이터의 모양 출력하기
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1395, 49, 1)
(599, 49, 1)
(1395, 46)
(599, 46)


# RNN

In [4]:
# 기본 RNN 모델을 구현하기 위한 함수
def vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(50, input_shape = (49,1), return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [5]:
model = KerasClassifier(build_fn = vanilla_rnn, epochs = 200, batch_size = 50, verbose = 1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)
print(accuracy_score(y_pred, y_test_))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Epoch 1/200
1395/1395 [==============================] - 1s 712us/step - loss: 3.0825 - accuracy: 0.4366
Epoch 2/200
1395/1395 [==============================] - 1s 497us/step - loss: 1.4056 - accuracy: 0.7147
Epoch 3/200
1395/1395 [==============================] - 1s 477us/step - loss: 1.2028 - accuracy: 0.7147
Epoch 4/200
1395/1395 [==============================] - 1s 505us/step - loss: 1.1793 - accuracy: 0.7147
Epoch 5/200
1395/1395 [==============================] - 1s 487us/step - loss: 1.1639 - accuracy: 0.7147
Epoch 6/200
1395/1395 [==============================] - 1s 488us/step - loss: 1.1621 - accuracy: 0.7147
Epoch 7/200
1395/1395 [==============================] - 1s 509us/step - loss: 1.1554 - accuracy: 0.7147
Epoch 8/200
1395/1395 [==============================] - 1s 474us/step - loss: 1.1537 - accuracy: 0.7147
Epoch 9/200
1395/1395 [==============================] - 1s 475us/step - loss:

1395/1395 [==============================] - 1s 482us/step - loss: 0.9311 - accuracy: 0.7341
Epoch 74/200
1395/1395 [==============================] - 1s 468us/step - loss: 0.9283 - accuracy: 0.7319
Epoch 75/200
1395/1395 [==============================] - 1s 478us/step - loss: 0.9204 - accuracy: 0.7312
Epoch 76/200
1395/1395 [==============================] - 1s 467us/step - loss: 0.9176 - accuracy: 0.7290
Epoch 77/200
1395/1395 [==============================] - 1s 481us/step - loss: 0.9139 - accuracy: 0.7269
Epoch 78/200
1395/1395 [==============================] - 1s 475us/step - loss: 0.9133 - accuracy: 0.7283
Epoch 79/200
1395/1395 [==============================] - 1s 470us/step - loss: 0.9058 - accuracy: 0.7355
Epoch 80/200
1395/1395 [==============================] - 1s 470us/step - loss: 0.9096 - accuracy: 0.7355
Epoch 81/200
1395/1395 [==============================] - 1s 471us/step - loss: 1.0407 - accuracy: 0.7118
Epoch 82/200
1395/1395 [==============================] - 1

Epoch 150/200
1395/1395 [==============================] - 1s 473us/step - loss: 0.9519 - accuracy: 0.7254
Epoch 151/200
1395/1395 [==============================] - 1s 496us/step - loss: 0.9388 - accuracy: 0.7190
Epoch 152/200
1395/1395 [==============================] - 1s 510us/step - loss: 0.9266 - accuracy: 0.7204
Epoch 153/200
1395/1395 [==============================] - 1s 482us/step - loss: 0.9193 - accuracy: 0.7233
Epoch 154/200
1395/1395 [==============================] - 1s 470us/step - loss: 0.9359 - accuracy: 0.7190
Epoch 155/200
1395/1395 [==============================] - 1s 485us/step - loss: 0.9075 - accuracy: 0.7269
Epoch 156/200
1395/1395 [==============================] - 1s 473us/step - loss: 0.9006 - accuracy: 0.7333
Epoch 157/200
1395/1395 [==============================] - 1s 473us/step - loss: 0.9092 - accuracy: 0.7283
Epoch 158/200
1395/1395 [==============================] - 1s 471us/step - loss: 0.8973 - accuracy: 0.7297
Epoch 159/200
1395/1395 [============

# Stacked RNN

In [6]:
# 기본 RNN 모델을 구현하기 위한 함수
def stacked_vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(50, input_shape = (49,1), return_sequences = True))   # return_sequences parameter has to be set True to stack
    model.add(SimpleRNN(50, return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [7]:
model = KerasClassifier(build_fn = stacked_vanilla_rnn, epochs = 200, batch_size = 50, verbose = 1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)
print(accuracy_score(y_pred, y_test_))

Epoch 1/200
1395/1395 [==============================] - 2s 1ms/step - loss: 2.1520 - accuracy: 0.5778
Epoch 2/200
1395/1395 [==============================] - 1s 965us/step - loss: 1.2274 - accuracy: 0.7147
Epoch 3/200
1395/1395 [==============================] - 1s 956us/step - loss: 1.1842 - accuracy: 0.7147
Epoch 4/200
1395/1395 [==============================] - 1s 973us/step - loss: 1.1715 - accuracy: 0.7147
Epoch 5/200
1395/1395 [==============================] - 1s 968us/step - loss: 1.1614 - accuracy: 0.7147
Epoch 6/200
1395/1395 [==============================] - 1s 944us/step - loss: 1.1648 - accuracy: 0.7147
Epoch 7/200
1395/1395 [==============================] - 1s 945us/step - loss: 1.1562 - accuracy: 0.7147
Epoch 8/200
1395/1395 [==============================] - 1s 945us/step - loss: 1.1520 - accuracy: 0.7147
Epoch 9/200
1395/1395 [==============================] - 1s 966us/step - loss: 1.1494 - accuracy: 0.7147
Epoch 10/200
1395/1395 [==============================] -

1395/1395 [==============================] - 1s 911us/step - loss: 0.9901 - accuracy: 0.7147
Epoch 79/200
1395/1395 [==============================] - 1s 1ms/step - loss: 0.9932 - accuracy: 0.7154
Epoch 80/200
1395/1395 [==============================] - 1s 942us/step - loss: 0.9911 - accuracy: 0.7118
Epoch 81/200
1395/1395 [==============================] - 1s 929us/step - loss: 0.9865 - accuracy: 0.7183
Epoch 82/200
1395/1395 [==============================] - 1s 986us/step - loss: 0.9853 - accuracy: 0.7183
Epoch 83/200
1395/1395 [==============================] - 1s 947us/step - loss: 0.9844 - accuracy: 0.7168
Epoch 84/200
1395/1395 [==============================] - 1s 926us/step - loss: 0.9808 - accuracy: 0.7125
Epoch 85/200
1395/1395 [==============================] - 1s 912us/step - loss: 0.9817 - accuracy: 0.7240
Epoch 86/200
1395/1395 [==============================] - 1s 991us/step - loss: 0.9772 - accuracy: 0.7190
Epoch 87/200
1395/1395 [==============================] - 1s 

1395/1395 [==============================] - 1s 935us/step - loss: 0.9017 - accuracy: 0.7355
Epoch 155/200
1395/1395 [==============================] - 1s 935us/step - loss: 0.8989 - accuracy: 0.7369
Epoch 156/200
1395/1395 [==============================] - 1s 968us/step - loss: 0.8962 - accuracy: 0.7326
Epoch 157/200
1395/1395 [==============================] - 1s 929us/step - loss: 0.8900 - accuracy: 0.7398
Epoch 158/200
1395/1395 [==============================] - 1s 901us/step - loss: 0.8904 - accuracy: 0.7369
Epoch 159/200
1395/1395 [==============================] - 1s 906us/step - loss: 0.8860 - accuracy: 0.7341
Epoch 160/200
1395/1395 [==============================] - 1s 908us/step - loss: 0.8866 - accuracy: 0.7341
Epoch 161/200
1395/1395 [==============================] - 1s 901us/step - loss: 0.8752 - accuracy: 0.7405
Epoch 162/200
1395/1395 [==============================] - 1s 902us/step - loss: 0.8754 - accuracy: 0.7348
Epoch 163/200
1395/1395 [==========================

# LSTM

In [8]:
# 단층 LSTM을 구현하기 위한 함수
from keras.layers import LSTM

def lstm():
    model = Sequential()
    model.add(LSTM(50, input_shape = (49,1), return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [9]:
model = KerasClassifier(build_fn = lstm, epochs = 200, batch_size = 50, verbose = 1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)
print(accuracy_score(y_pred, y_test_))

Epoch 1/200
1395/1395 [==============================] - 1s 829us/step - loss: 2.9303 - accuracy: 0.6394
Epoch 2/200
1395/1395 [==============================] - 1s 691us/step - loss: 1.3426 - accuracy: 0.7147
Epoch 3/200
1395/1395 [==============================] - 1s 672us/step - loss: 1.1883 - accuracy: 0.7147
Epoch 4/200
1395/1395 [==============================] - 1s 683us/step - loss: 1.1667 - accuracy: 0.7147
Epoch 5/200
1395/1395 [==============================] - 1s 687us/step - loss: 1.1583 - accuracy: 0.7147
Epoch 6/200
1395/1395 [==============================] - 1s 704us/step - loss: 1.1510 - accuracy: 0.7147
Epoch 7/200
1395/1395 [==============================] - 1s 680us/step - loss: 1.1429 - accuracy: 0.7147
Epoch 8/200
1395/1395 [==============================] - 1s 674us/step - loss: 1.1309 - accuracy: 0.7147
Epoch 9/200
1395/1395 [==============================] - 1s 689us/step - loss: 1.1124 - accuracy: 0.7147
Epoch 10/200
1395/1395 [==============================]

1395/1395 [==============================] - 1s 682us/step - loss: 0.6372 - accuracy: 0.8430
Epoch 78/200
1395/1395 [==============================] - 1s 690us/step - loss: 0.6248 - accuracy: 0.8437
Epoch 79/200
1395/1395 [==============================] - 1s 679us/step - loss: 0.6188 - accuracy: 0.8473
Epoch 80/200
1395/1395 [==============================] - 1s 680us/step - loss: 0.6257 - accuracy: 0.8444
Epoch 81/200
1395/1395 [==============================] - 1s 689us/step - loss: 0.6228 - accuracy: 0.8452
Epoch 82/200
1395/1395 [==============================] - 1s 685us/step - loss: 0.6134 - accuracy: 0.8509
Epoch 83/200
1395/1395 [==============================] - 1s 684us/step - loss: 0.6146 - accuracy: 0.8452
Epoch 84/200
1395/1395 [==============================] - 1s 685us/step - loss: 0.6270 - accuracy: 0.8509
Epoch 85/200
1395/1395 [==============================] - 1s 684us/step - loss: 0.6262 - accuracy: 0.8487
Epoch 86/200
1395/1395 [==============================] - 1

1395/1395 [==============================] - 1s 678us/step - loss: 0.4286 - accuracy: 0.8896
Epoch 154/200
1395/1395 [==============================] - 1s 685us/step - loss: 0.4245 - accuracy: 0.8953
Epoch 155/200
1395/1395 [==============================] - 1s 685us/step - loss: 0.4226 - accuracy: 0.8918
Epoch 156/200
1395/1395 [==============================] - 1s 695us/step - loss: 0.4134 - accuracy: 0.8968
Epoch 157/200
1395/1395 [==============================] - 1s 685us/step - loss: 0.4242 - accuracy: 0.8932
Epoch 158/200
1395/1395 [==============================] - 1s 694us/step - loss: 0.4107 - accuracy: 0.8968
Epoch 159/200
1395/1395 [==============================] - 1s 680us/step - loss: 0.4038 - accuracy: 0.9011
Epoch 160/200
1395/1395 [==============================] - 1s 691us/step - loss: 0.4085 - accuracy: 0.8996
Epoch 161/200
1395/1395 [==============================] - 1s 683us/step - loss: 0.4006 - accuracy: 0.8989
Epoch 162/200
1395/1395 [==========================

# Deep LSTM

In [10]:
# 기본 RNN 모델을 구현하기 위한 함수
def deep_lstm():
    model = Sequential()
    model.add(LSTM(20, input_shape = (49,1), return_sequences = True))
    model.add(LSTM(20, return_sequences = True))
    model.add(LSTM(20, return_sequences = True))
    model.add(LSTM(20, return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [11]:
model = KerasClassifier(build_fn = deep_lstm, epochs = 200, batch_size = 50, verbose = 1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)
print(accuracy_score(y_pred, y_test_))

Epoch 1/200
1395/1395 [==============================] - 4s 3ms/step - loss: 3.1941 - accuracy: 0.5018
Epoch 2/200
1395/1395 [==============================] - 4s 3ms/step - loss: 1.8774 - accuracy: 0.7147
Epoch 3/200
1395/1395 [==============================] - 4s 3ms/step - loss: 1.3746 - accuracy: 0.7147
Epoch 4/200
1395/1395 [==============================] - 4s 3ms/step - loss: 1.2490 - accuracy: 0.7147
Epoch 5/200
1395/1395 [==============================] - 4s 3ms/step - loss: 1.2108 - accuracy: 0.7147
Epoch 6/200
1395/1395 [==============================] - 4s 3ms/step - loss: 1.1936 - accuracy: 0.7147
Epoch 7/200
1395/1395 [==============================] - 4s 3ms/step - loss: 1.1833 - accuracy: 0.7147
Epoch 8/200
1395/1395 [==============================] - 4s 3ms/step - loss: 1.1765 - accuracy: 0.7147
Epoch 9/200
1395/1395 [==============================] - 4s 3ms/step - loss: 1.1726 - accuracy: 0.7147
Epoch 10/200
1395/1395 [==============================] - 4s 3ms/step - l

1395/1395 [==============================] - 4s 3ms/step - loss: 0.7698 - accuracy: 0.8014
Epoch 80/200
1395/1395 [==============================] - 4s 3ms/step - loss: 0.7686 - accuracy: 0.8022
Epoch 81/200
1395/1395 [==============================] - 4s 3ms/step - loss: 0.7535 - accuracy: 0.8036
Epoch 82/200
1395/1395 [==============================] - 4s 3ms/step - loss: 0.7658 - accuracy: 0.7993
Epoch 83/200
1395/1395 [==============================] - 4s 3ms/step - loss: 0.7547 - accuracy: 0.8007
Epoch 84/200
1395/1395 [==============================] - 4s 3ms/step - loss: 0.7556 - accuracy: 0.8043
Epoch 85/200
1395/1395 [==============================] - 4s 3ms/step - loss: 0.7856 - accuracy: 0.7978
Epoch 86/200
1395/1395 [==============================] - 4s 3ms/step - loss: 0.7543 - accuracy: 0.8022
Epoch 87/200
1395/1395 [==============================] - 4s 3ms/step - loss: 1.0359 - accuracy: 0.6724
Epoch 88/200
1395/1395 [==============================] - 4s 3ms/step - loss:

1395/1395 [==============================] - 4s 3ms/step - loss: 0.6755 - accuracy: 0.8158
Epoch 158/200
1395/1395 [==============================] - 4s 3ms/step - loss: 0.6738 - accuracy: 0.8100
Epoch 159/200
1395/1395 [==============================] - 4s 3ms/step - loss: 0.6681 - accuracy: 0.8151
Epoch 160/200
1395/1395 [==============================] - 4s 3ms/step - loss: 0.6733 - accuracy: 0.8122
Epoch 161/200
1395/1395 [==============================] - 4s 3ms/step - loss: 0.6620 - accuracy: 0.8100
Epoch 162/200
1395/1395 [==============================] - 4s 3ms/step - loss: 0.7631 - accuracy: 0.8007
Epoch 163/200
1395/1395 [==============================] - 4s 3ms/step - loss: 0.7191 - accuracy: 0.8043
Epoch 164/200
1395/1395 [==============================] - 4s 3ms/step - loss: 0.6806 - accuracy: 0.8065
Epoch 165/200
1395/1395 [==============================] - 4s 3ms/step - loss: 0.6794 - accuracy: 0.8122
Epoch 166/200
1395/1395 [==============================] - 4s 3ms/ste

# Bidirectional RNN

In [12]:
# 양방향 RNN을 구현하기 위한 함수
def bidirectional_lstm():
    model = Sequential()
    model.add(Bidirectional(LSTM(20, return_sequences = False), input_shape = (49,1)))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [13]:
model = KerasClassifier(build_fn = bidirectional_lstm, epochs = 200, batch_size = 50, verbose = 1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)
print(accuracy_score(y_pred, y_test_))

Epoch 1/200
1395/1395 [==============================] - 2s 1ms/step - loss: 3.3732 - accuracy: 0.3735
Epoch 2/200
1395/1395 [==============================] - 1s 987us/step - loss: 2.3309 - accuracy: 0.7147
Epoch 3/200
1395/1395 [==============================] - 1s 1ms/step - loss: 1.3602 - accuracy: 0.7147
Epoch 4/200
1395/1395 [==============================] - 1s 1ms/step - loss: 1.1032 - accuracy: 0.7147
Epoch 5/200
1395/1395 [==============================] - 1s 1ms/step - loss: 1.0260 - accuracy: 0.7147
Epoch 6/200
1395/1395 [==============================] - 1s 1ms/step - loss: 0.9827 - accuracy: 0.7161
Epoch 7/200
1395/1395 [==============================] - 1s 1ms/step - loss: 0.9494 - accuracy: 0.7534
Epoch 8/200
1395/1395 [==============================] - 1s 1ms/step - loss: 0.9273 - accuracy: 0.7713: 0s - loss: 0.9570 - accura
Epoch 9/200
1395/1395 [==============================] - 1s 1000us/step - loss: 0.9042 - accuracy: 0.7792
Epoch 10/200
1395/1395 [================

1395/1395 [==============================] - 1s 1ms/step - loss: 0.6020 - accuracy: 0.8495
Epoch 79/200
1395/1395 [==============================] - 1s 1ms/step - loss: 0.5908 - accuracy: 0.8523
Epoch 80/200
1395/1395 [==============================] - 1s 1ms/step - loss: 0.5929 - accuracy: 0.8545
Epoch 81/200
1395/1395 [==============================] - 1s 996us/step - loss: 0.5905 - accuracy: 0.8552
Epoch 82/200
1395/1395 [==============================] - 1s 998us/step - loss: 0.5949 - accuracy: 0.8502
Epoch 83/200
1395/1395 [==============================] - 1s 1ms/step - loss: 0.5854 - accuracy: 0.8552
Epoch 84/200
1395/1395 [==============================] - 1s 1ms/step - loss: 0.5840 - accuracy: 0.8538
Epoch 85/200
1395/1395 [==============================] - 1s 990us/step - loss: 0.6009 - accuracy: 0.8523
Epoch 86/200
1395/1395 [==============================] - 1s 1ms/step - loss: 0.5843 - accuracy: 0.8566
Epoch 87/200
1395/1395 [==============================] - 1s 991us/step

1395/1395 [==============================] - 1s 997us/step - loss: 0.4892 - accuracy: 0.8710
Epoch 156/200
1395/1395 [==============================] - 1s 1ms/step - loss: 0.4882 - accuracy: 0.8681
Epoch 157/200
1395/1395 [==============================] - 1s 1ms/step - loss: 0.5031 - accuracy: 0.8667
Epoch 158/200
1395/1395 [==============================] - 1s 1ms/step - loss: 0.4910 - accuracy: 0.8688
Epoch 159/200
1395/1395 [==============================] - 1s 990us/step - loss: 0.4963 - accuracy: 0.8652
Epoch 160/200
1395/1395 [==============================] - 1s 1ms/step - loss: 0.5082 - accuracy: 0.8659
Epoch 161/200
1395/1395 [==============================] - 1s 1ms/step - loss: 0.4987 - accuracy: 0.8681
Epoch 162/200
1395/1395 [==============================] - 1s 1ms/step - loss: 0.4829 - accuracy: 0.8717
Epoch 163/200
1395/1395 [==============================] - 1s 998us/step - loss: 0.4815 - accuracy: 0.8738
Epoch 164/200
1395/1395 [==============================] - 1s 1

# Deep Bidirectional RNN

In [14]:
# 다층 양방향 LSTM을 구현하기 위한 함수
def deep_bidirectional_lstm():
    model = Sequential()
    model.add(Bidirectional(LSTM(10, return_sequences = True), input_shape = (49,1)))
    model.add(Bidirectional(LSTM(10, return_sequences = True)))
    model.add(Bidirectional(LSTM(10, return_sequences = True)))
    model.add(Bidirectional(LSTM(10, return_sequences = False)))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [15]:
model = KerasClassifier(build_fn = deep_bidirectional_lstm, epochs = 200, batch_size = 50, verbose = 1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)
print(accuracy_score(y_pred, y_test_))

Epoch 1/200
1395/1395 [==============================] - 7s 5ms/step - loss: 3.4333 - accuracy: 0.5505
Epoch 2/200
1395/1395 [==============================] - 6s 4ms/step - loss: 2.2575 - accuracy: 0.7147
Epoch 3/200
1395/1395 [==============================] - 6s 4ms/step - loss: 1.4730 - accuracy: 0.7147
Epoch 4/200
1395/1395 [==============================] - 6s 4ms/step - loss: 1.2506 - accuracy: 0.7147
Epoch 5/200
1395/1395 [==============================] - 6s 4ms/step - loss: 1.2097 - accuracy: 0.7147
Epoch 6/200
1395/1395 [==============================] - 6s 4ms/step - loss: 1.1913 - accuracy: 0.7147
Epoch 7/200
1395/1395 [==============================] - 6s 4ms/step - loss: 1.1805 - accuracy: 0.7147
Epoch 8/200
1395/1395 [==============================] - 6s 4ms/step - loss: 1.1735 - accuracy: 0.7147
Epoch 9/200
1395/1395 [==============================] - 6s 4ms/step - loss: 1.1695 - accuracy: 0.7147
Epoch 10/200
1395/1395 [==============================] - 6s 4ms/step - l

1395/1395 [==============================] - 6s 4ms/step - loss: 0.6263 - accuracy: 0.8581
Epoch 80/200
1395/1395 [==============================] - 6s 4ms/step - loss: 0.6383 - accuracy: 0.8509
Epoch 81/200
1395/1395 [==============================] - 6s 4ms/step - loss: 0.6180 - accuracy: 0.8573
Epoch 82/200
1395/1395 [==============================] - 6s 4ms/step - loss: 0.6057 - accuracy: 0.8652
Epoch 83/200
1395/1395 [==============================] - 6s 4ms/step - loss: 0.6143 - accuracy: 0.8609
Epoch 84/200
1395/1395 [==============================] - 6s 4ms/step - loss: 0.6279 - accuracy: 0.8538
Epoch 85/200
1395/1395 [==============================] - 6s 4ms/step - loss: 0.6224 - accuracy: 0.8581
Epoch 86/200
1395/1395 [==============================] - 6s 4ms/step - loss: 0.6096 - accuracy: 0.8631
Epoch 87/200
1395/1395 [==============================] - 6s 4ms/step - loss: 0.6202 - accuracy: 0.8581
Epoch 88/200
1395/1395 [==============================] - 6s 4ms/step - loss:

1395/1395 [==============================] - 6s 4ms/step - loss: 0.5832 - accuracy: 0.8495
Epoch 158/200
1395/1395 [==============================] - 6s 4ms/step - loss: 0.5578 - accuracy: 0.8595
Epoch 159/200
1395/1395 [==============================] - 6s 4ms/step - loss: 0.5798 - accuracy: 0.8523
Epoch 160/200
1395/1395 [==============================] - 6s 4ms/step - loss: 0.5639 - accuracy: 0.8631
Epoch 161/200
1395/1395 [==============================] - 6s 4ms/step - loss: 0.5253 - accuracy: 0.8760
Epoch 162/200
1395/1395 [==============================] - 6s 4ms/step - loss: 0.5113 - accuracy: 0.8803
Epoch 163/200
1395/1395 [==============================] - 6s 4ms/step - loss: 0.5171 - accuracy: 0.8738
Epoch 164/200
1395/1395 [==============================] - 6s 4ms/step - loss: 0.5124 - accuracy: 0.8753
Epoch 165/200
1395/1395 [==============================] - 6s 4ms/step - loss: 0.5036 - accuracy: 0.8746
Epoch 166/200
1395/1395 [==============================] - 6s 4ms/ste